### Throughout this exercise, you may need to refer to online documentation.

Pandas: http://pandas.pydata.org/pandas-docs/stable/

Numpy & Scipy: https://docs.scipy.org/doc/

In [1]:
# First, import packages and load some data
import pandas      as pd
import numpy       as np
import scipy       as sp
import scipy.stats as stats
from statistics import mean

datafile = './data/pima-indians-diabetes.csv'

df = pd.read_csv(datafile, header=0, index_col=None)
# df.head()

## Replace 0 with nan in appropriate columns 

In [25]:
replace_dict = {'plasma_glucose_concentration': 0, 'diastolic_blood_pressure': 0, 'triceps_thickness': 0, '2-hour_serum_insulin': 0, 'BMI': 0, 'diabetes_pedigreen': 0, 'age': 0}

#replace 0 values with nan for columns in replace_dict
df.replace(replace_dict, np.nan, inplace=True)

#check min values for selected columns no longe 0.0
df.describe()

,times_pregnant,plasma_glucose_concentration,diastolic_blood_pressure,triceps_thickness,2-hour_serum_insulin,BMI,diabetes_pedigreen,age,diabetes
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [111]:
# Get the count (of all non-NaNs), min, max, mean, median and mode of each column.

#set index and labels
index = ['Mean', 'Median', 'Min', 'Max', 'Non NaN Values']
labels = ['times_pregnant', 'plasma_glucose_concentration', 'diastolic_blood_pressure', 'triceps_thickness', '2-hour_serum_insulin', 'BMI', 'diabetes_pedigreen', 'age', 'diabetes']

#get all stats values that we can with df built in funcs
means = df.mean(axis=0, skipna=True) #=> describe already skips na
medians = df.median(axis=0, skipna=True)
mins = df.min(axis=0, skipna=True)
maxs = df.max(axis=0, skipna=True)
not_na_counts = df.notnull().sum()

#create stats df
statdf = pd.DataFrame([means, medians, mins, maxs, not_na_counts], index=index)

#get mode using stats.mode function
mode_result = stats.mode(df, nan_policy='omit')
mode_arr = mode_result[0][0]

# create mode df to add to other stats df
modes = pd.DataFrame([[1, 99, 70, 32, 105, 32, 0.254, 22, 0]], columns=labels, index=['Mode'])

#join mode df to stats df
statdf.append(modes)

# Which column has the most missing values?
    # just look at non-null count and find min
    # in this case the 2-hour serum insulin column has the most missing values


,times_pregnant,plasma_glucose_concentration,diastolic_blood_pressure,triceps_thickness,2-hour_serum_insulin,BMI,diabetes_pedigreen,age,diabetes
Mean,3.845052,121.686763,72.405184,29.15342,155.548223,32.457464,0.471876,33.240885,0.348958
Median,3.000000,117.000000,72.000000,29.00000,125.000000,32.300000,0.372500,29.000000,0.000000
Min,0.000000,44.000000,24.000000,7.00000,14.000000,18.200000,0.078000,21.000000,0.000000
Max,17.000000,199.000000,122.000000,99.00000,846.000000,67.100000,2.420000,81.000000,1.000000
Non NaN Values,768.000000,763.000000,733.000000,541.00000,394.000000,757.000000,768.000000,768.000000,768.000000
Mode,1.000000,99.000000,70.000000,32.00000,105.000000,32.000000,0.254000,22.000000,0.000000


## Calculate z-scores

In [3]:
def centre(col):
    array_vals = col.to_numpy()
    centered_vals = stats.zscore(array_vals, nan_policy='omit')
    return centered_vals


def check_z_transform(values):
    values = np.array(values)
    mean = round(np.nanmean(values), 2)
    std = np.nanstd(values)
    print('Mean: ', mean, '\nStd: ', std)

# run a test on a column to check mean and std of z-scores
centered_plasma = centre(df['plasma_glucose_concentration'])
check_z_transform(centered_plasma)


Mean:  -0.0 
Std:  1.0


## Apply center function to DataFrame

In [7]:
# Make a Copy
cdf = df.copy()
# Apply the centre function to the dataset
cdf = cdf.apply(centre)
# round data to make describe easier to visually check that means and stds are about 0 and 1 respectively
cdf.round(3).describe()

,times_pregnant,plasma_glucose_concentration,diastolic_blood_pressure,triceps_thickness,2-hour_serum_insulin,BMI,diabetes_pedigreen,age,diabetes
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,-0.000020,0.000003,-0.000046,0.000081,-0.000055,-0.000013,0.000030,0.000046,0.000115
std,1.000765,1.000674,1.000727,1.000570,1.000693,1.000620,1.000651,1.000611,1.000644
min,-1.142000,-3.784000,-3.573000,-1.288000,-0.693000,-4.060000,-1.190000,-1.042000,-0.732000
25%,-0.845000,-0.685000,-0.367000,-1.288000,-0.693000,-0.596000,-0.688750,-0.786000,-0.732000
50%,-0.251000,-0.122000,0.150000,0.155000,-0.428000,0.001000,-0.300500,-0.361000,-0.732000
75%,0.640000,0.605750,0.563000,0.719000,0.412250,0.585000,0.465750,0.660000,1.366000
max,3.907000,2.444000,2.735000,4.922000,6.653000,4.456000,5.884000,4.064000,1.366000


## Get min and max of centered dataset

In [8]:
# Finally! Let's look for outliers by getting the min and max in the new centred df:

mins = cdf.min(axis=0, skipna=True)
maxs = cdf.max(axis=0, skipna=True)
means = cdf.mean(axis=0, skipna=True)
statcdf = pd.DataFrame([mins, maxs, means], index=['Min', 'Max', 'Mean'])
statcdf



,times_pregnant,plasma_glucose_concentration,diastolic_blood_pressure,triceps_thickness,2-hour_serum_insulin,BMI,diabetes_pedigreen,age,diabetes
Min,-1.141852e+00,-3.783654e+00,-3.572597e+00,-1.288212e+00,-6.928906e-01,-4.060474e+00,-1.189553e+00,-1.041549e+00,-7.321202e-01
Max,3.906578e+00,2.444478e+00,2.734528e+00,4.921866e+00,6.652839e+00,4.455807e+00,5.883565e+00,4.063716e+00,1.365896e+00
Mean,-6.476301e-17,-9.251859e-18,1.503427e-17,1.006140e-16,-3.006854e-17,2.590520e-16,2.451743e-16,1.931325e-16,7.401487e-17


### trick question: What is the mean of cdf['times_pregnant']? Why isn't it zero?

The mean of times_pregnant is -6.47e-17 which is approximately 0.   
Not sure if this is what this question is getting at.
